# Experiment with Matplotlib, OpenCV, and Pillow

* Use inputs from matplotlib, OpenCV, and Pillow as texture maps in a 3D Sketch
* Easily convert Pillow images, matplotlib figures, and numpy arrays to Py5Image objects

In [ ]:
# the following code is necessary for MacOS users
import sys

if sys.platform == 'darwin':
    # Equivalent to `%gui osx`
    get_ipython().run_line_magic('gui', 'osx')

In [ ]:
from collections import deque

import cv2
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

import py5

In [ ]:
CUBE_SIZE = 175

# Load py5 Logo with Pillow

In [ ]:
logo_image = Image.open("data/logo.jpg")


# Code to Plot Sketch frame_rate with matplotlib

In [ ]:
matplotlib.use('agg')
matplotlib.style.use(['ggplot', 'fast'])

In [ ]:
frame_rates = deque([60] * 250, maxlen=250)


def create_chart(frame_rates):
    figure, ax1 = plt.subplots(1, 1, figsize=(5, 5))
    (line,) = ax1.plot(range(-len(frame_rates), 0), frame_rates)
    ax1.set_ylim(0, 70)
    ax1.set_xlabel("framerate observations")
    ax1.set_ylabel("framerate per second")
    ax1.set_title("py5 sketch framerate")

    return figure, line


def update_chart(frame_rates, line, figure, chart):
    # update line object in existing figure
    # this is much faster than creating a new figure each time
    line.set_ydata(frame_rates)
    py5.convert_image(figure, dst=chart)


# Read Webcam with OpenCV

In [ ]:
cap = cv2.VideoCapture(0)
webcam_image = None


def read_webcam():
    global webcam_image
    ret, frame = cap.read()
    if frame is not None:
        # center crop
        h, w, _ = frame.shape
        d = (w - h) // 2
        frame2 = frame[:, d : (w - d), ::-1].copy()

        if webcam_image is None:
            # create new webcam_image object
            webcam_image = py5.create_image_from_numpy(frame2, bands="RGB")
        else:
            # recycle webcam_image object
            webcam_image.np_pixels[:, :, 1:] = frame2
            webcam_image.update_np_pixels()


# Sketch Code

In [ ]:
def draw_cube_face(img, x, y, z):
    with py5.push_matrix():
        py5.rotate_x(py5.radians(x))
        py5.rotate_y(py5.radians(y))
        py5.rotate_z(py5.radians(z))

        with py5.begin_shape():
            py5.texture(img)
            py5.vertex(-CUBE_SIZE, -CUBE_SIZE, CUBE_SIZE, 0, 0)
            py5.vertex(CUBE_SIZE, -CUBE_SIZE, CUBE_SIZE, 1, 0)
            py5.vertex(CUBE_SIZE, CUBE_SIZE, CUBE_SIZE, 1, 1)
            py5.vertex(-CUBE_SIZE, CUBE_SIZE, CUBE_SIZE, 0, 1)


In [ ]:
x_rot = 0
y_rot = 0
z_rot = 0


def setup():
    global chart, logo
    py5.size(750, 750, py5.P3D)
    py5.rect_mode(py5.CENTER)
    py5.no_stroke()
    py5.texture_mode(py5.NORMAL)
    py5.get_surface().set_always_on_top(True)

    # prepare logo
    logo = py5.convert_image(logo_image)

    # prepare chart
    figure, line = create_chart(frame_rates)
    chart = py5.convert_image(figure)
    # start chart update thread
    py5.launch_repeating_thread(update_chart, args=(frame_rates, line, figure, chart))

    # start webcam thread
    py5.launch_repeating_thread(read_webcam, name="webcam")


def draw():
    global x_rot, y_rot, z_rot

    x_rot += 0.4
    y_rot += 0.5
    z_rot += 0.6

    # record new frame_rate observation
    frame_rates.append(py5.get_frame_rate())

    py5.background(0)

    py5.translate(py5.width / 2, py5.height / 2, 0)
    py5.rotate_x(py5.radians(x_rot))
    py5.rotate_y(py5.radians(y_rot))
    py5.rotate_z(py5.radians(z_rot))

    draw_cube_face(webcam_image, 0, 0, 0)
    draw_cube_face(webcam_image, 0, 180, 0)
    draw_cube_face(chart, 0, 90, 0)
    draw_cube_face(chart, 0, 270, 0)
    draw_cube_face(chart, 90, 0, 0)
    draw_cube_face(logo, -90, 0, 0)


def exiting():
    # always release the webcam!
    cap.release()


In [ ]:
py5.run_sketch()


# Summary

* py5 can easily convert matplotlib figures, Pillow Images, and numpy arrays to image objects it can use for any Sketch
* py5 has builtin support for launching threads, aiding Sketch performance
* Putting slow or potentially blocking code in separate threads is important for achieving a good frame rate
* Demoing silly examples at PyCon makes presenting more fun